In [ ]:
#نادين اشرف كمال مغربي
#هبة الله محمد ممدوح
#ريم وليد علي الشناوي
#مصطفي عادل ذكريا حسن
#مريم ايمن محمد السيد

In [ ]:
import numpy as np
import pandas as pd
import itertools
from itertools import permutations
import io
from google.colab import files
uploaded = files.upload()
BlosumMatrix = pd.read_csv(io.BytesIO(uploaded['blosum62.csv']))
print(BlosumMatrix)

In [ ]:
def BlosumDictionary(matrix):
    AminoAcids_Scores=BlosumMatrix.values.tolist() #convert to list that contains 24 lists
    charVecs = [] #empty list
    for ch in range(0,2):
        charVecs.append(list(matrix.index))
    AminoAcidLetters1 = charVecs[0] #first letters in csv file
    AminoAcidLetters2 = charVecs[1] #second letters in csv file
    AminoAcidCombined= list(itertools.product(AminoAcidLetters1,AminoAcidLetters2)) #all possible combinations
    K = 24 #rows in csv file
    AminoAcidKeys= []
    for idx in range(0, K):#to make the amino acid list of list
        AminoAcidKeys.append( AminoAcidCombined[idx::K]) #reverse combination
    BlosumDictionary=dict(zip(AminoAcidKeys[0],AminoAcids_Scores[0]))#?????????
    for i in range(1,24):
        IteratedDictionary= dict(zip(AminoAcidKeys[i],AminoAcids_Scores[i]))
        BlosumDictionary.update(IteratedDictionary)
    return BlosumDictionary

In [ ]:
blosumScore=BlosumDictionary(BlosumMatrix)

In [ ]:
def split(string):
  return list(string)

In [ ]:
def tracebackDNA(Matrix,rowNum,colNum,seq1,seq2):
    GAFirst =""
    GASecond=""
    GAMatch=""
    row = rowNum #row
    column = colNum #col
    while row > 0 and column > 0:
        up = Matrix[row-1][column]-1
        left=Matrix[row][column-1]-1
        if  seq1[column-1]==seq2[row-1]:
            score=1
        else:
            score=-2
        diagonal = Matrix[row-1][column-1]+score
        if Matrix[row][column]==diagonal:
            GAFirst+=seq1[column-1] #adds letters from first sequence
            GASecond+=seq2[row-1] #adds letters from second sequence
            if score==1:
                GAMatch+="|"
            else:
                GAMatch+=" "
            row-=1
            column-=1 #leave row and column
        elif Matrix[row][column] == up:
            GAFirst+="-"
            GAMatch+=" "
            GASecond+=seq2[row-1]
            row-=1 #leave row
        else:
            GAFirst+=seq1[column-1]
            GASecond+="-"
            GAMatch+=" "
            column-=1 #leave column

    while (row>0):
        GAFirst+="-"
        GASecond+=seq2[row-1]
        GAMatch+=" "
        row-=1
    while(column>0):
        GAFirst+=seq1[column-1]
        GASecond+="-"
        GAMatch+=" "
        column-=1

    GAFirst=GAFirst[::-1] #prints letters in correct order (reversed)
    GAMatch=GAMatch[::-1]
    GASecond=GASecond[::-1]
    print("Traceback of this DNA sequence:")
    print(GAFirst)
    print(GAMatch)
    print(GASecond)


In [ ]:
def tracebackProtein(Matrix,rowNum,colNum,seq1,seq2):
    GAFirst =""
    GASecond=""
    row = rowNum #row
    column = colNum #col
    if row > column:
              MinLength = column
    else:
              MinLength = row
    splitSeq1= split(seq1)
    splitSeq2= split(seq2)
    while row > 0 and column > 0:
          up = Matrix[row-1][column]-1
          left=Matrix[row][column-1]-1
          for x in range(MinLength):
            diagonal = Matrix[row-1][column-1]+blosumScore[(splitSeq2[row-1],splitSeq1[column-1])]
          if Matrix[row][column]==diagonal:
            GAFirst+=seq1[column-1]
            GASecond+=seq2[row-1]
            row-=1
            column-=1
          elif Matrix[row][column] == up:
            GAFirst+="-"
            GASecond+=seq2[row-1]
            row-=1
          else:
            GAFirst+=seq1[column-1]
            GASecond+="-"
            column-=1
    while (row>0):
        GAFirst+="-"
        GASecond+=seq2[row-1]
        row-=1
    while(column>0):
        GAFirst+=seq1[column-1]
        GASecond+="-"
        column-=1
    GAFirst=GAFirst[::-1]
    GASecond=GASecond[::-1]
    print("Traceback of this protein sequence:")
    print(GAFirst)
    print(GASecond)

In [ ]:
def globalAlignmentDNA(seq1, seq2):
    lengthOfFirstSeq=len(seq1) #what represents columns
    lengthOfSecondSeq=len(seq2) #what represents rows
    Matrix = [[0 for row in range(lengthOfFirstSeq+1)] for column in range(lengthOfSecondSeq+1)] #fill matrix with zeros
    print("Shape of Matrix is : "+str(np.shape(Matrix)))
    for row in range (lengthOfSecondSeq+1):
        Matrix[row][0]=-row #fixed column and adding gaps to every row in that column
    for column in range (lengthOfFirstSeq+1):
        Matrix[0][column] =-column #fixed row and adding gaps to every column in that row
    for row in range(1,lengthOfSecondSeq+1,1):
        for column in range(1,lengthOfFirstSeq+1,1):
            if seq1[column-1] == seq2[row-1]:  #to see if a letter matches the other and sets the score
                score = 1#matchvalue
            else :
                score =-2 #mismatchvalue
            Matrix[row][column] = max(Matrix[row-1][column]-1, Matrix[row][column-1]-1,Matrix[row-1][column-1]+score)
    for row in range (lengthOfSecondSeq+1):
            for column in range (lengthOfFirstSeq+1):
                print(Matrix[row][column],end="")
            print("\n")
    print("Score: ",Matrix[row][column])
    tracebackDNA(Matrix,row,column,seq1,seq2)


In [ ]:
def globalAlignmentProtein(Seq1,Seq2):
  lengthOfFirstSeq=len(Seq1)
  lengthOfSecondSeq=len(Seq2)
  Matrix = [[0 for row in range(lengthOfFirstSeq+1)] for columns in range(lengthOfSecondSeq+1)]
  print("Shape of Matrix is : "+str(np.shape(Matrix)))
  for row in range (lengthOfSecondSeq+1):
        Matrix[row][0]=-row #fixed column and adding gaps to every row in that column
  for columns in range (lengthOfFirstSeq+1):
        Matrix[0][columns]=-columns #fixed row and adding gaps to every column in that column
  splitSeq1= split(Seq1)
  splitSeq2= split(Seq2)
  for row in range(1,lengthOfSecondSeq+1,1):
      for column in range(1,lengthOfFirstSeq+1,1): #hena mesh ha2dr a7otha ashan kda hyw2f kolo mesh bs el diagonal
          Matrix[row][column] = max(Matrix[row-1][column]-1, Matrix[row][column-1]-1,Matrix[row-1][column-1]+blosumScore[(splitSeq1[column-1],splitSeq2[row-1])])
  for row in range (lengthOfSecondSeq+1):
            for col in range (lengthOfFirstSeq+1):
                print(Matrix[row][col],end="")
            print("\n")
  print("Score: ",Matrix[row][column])
  print("row: ",row, "column: ",column)
  tracebackProtein(Matrix,row,column,Seq1,Seq2)

In [ ]:
def correctFormat(Seq1,Seq2,seqCheck):
  correctSeq1=all(characters in seqCheck for characters in Seq1) #checks if all characters in entered sequence has the allowed
  correctSeq2=all(characters in seqCheck for characters in Seq2) #seqCheck sent from checkSequence function
  return correctSeq1 and correctSeq2 #sends true if both are correct

In [ ]:
def checkSequence(typeSeq):
  if typeSeq=="1":
    Seq1=input("Enter First Sequence ").upper()
    Seq2=input("Enter Second Sequence ").upper()
    DNAseq='ACTG'
    correct=correctFormat(Seq1,Seq2,DNAseq) #send the sequences and which sequence format it should have (DNA or Protein Letters)
    if(correct==True):
      globalAlignmentDNA(Seq1,Seq2)
    else:
      print('Invalid sequence')
      checkSequence(typeSeq)
  elif typeSeq=="2":
    Seq1=input("Enter First Sequence ").upper()
    Seq2=input("Enter Second Sequence ").upper()
    proteinSeq='ABCDEFGHIKLMNPQRSTVWY'
    correct=correctFormat(Seq1,Seq2,proteinSeq)
    if(correct==True):
        globalAlignmentProtein(Seq1,Seq2)
    else:
        print('Invalid sequence')
        checkSequence(typeSeq)
  else:
    print("Option isn't valid")

In [ ]:
print("Global ALignment")
typeSeq = input("1- DNA , 2-Amino Acid ")
checkSequence(typeSeq)